In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from neural_network.layer import DenseLayer
from neural_network.activation import Sigmoid
from neural_network.loss import MSE, RMSE
from neural_network.optimizer import SgdMomentum

### Checking Sigmoid

In [2]:
import numpy as np

# checking analytical and numerical gradients for sigmoid activation function
def func(x):
    return 1 / (1 + np.exp(-x))

def analytical_gradient(func, x):
    return func(x) * (1 - func(x))

def numerical_gradient(func, x, h=1e-9):
    x = np.asarray(x)
    return (func(x + h) - func(x)) / h

x = np.array([[2,4,6,8,10]])
analytical_grad = analytical_gradient(func, x)
numerical_grad = numerical_gradient(func, x)

# Compare the gradients
print("Analytical Gradient:\n", analytical_grad[:5])
print("\nNumerical Gradient:\n", numerical_grad[:5])


Analytical Gradient:
 [[1.04993585e-01 1.76627062e-02 2.46650929e-03 3.35237671e-04
  4.53958077e-05]]

Numerical Gradient:
 [[1.04993680e-01 1.76625381e-02 2.46647147e-03 3.35065309e-04
  4.52970994e-05]]


In [3]:
# checking analytical and numerical gradients for squared function
import numpy as np

def func(x):
    return x**2

def analytical_gradient(func, x):
    return 2*x

def numerical_gradient(func, x, h=1e-9):
    x = np.asarray(x)
    return (func(x + h) - func(x)) / h

x = np.array([[2,4,6,8,10]])
analytical_grad = analytical_gradient(func, x)
numerical_grad = numerical_gradient(func, x)

# Compare the gradients
print("Analytical Gradient:\n", analytical_grad[:5])
print("\nNumerical Gradient:\n", numerical_grad[:5])


Analytical Gradient:
 [[ 4  8 12 16 20]]

Numerical Gradient:
 [[ 4.00000033  8.00000066 12.00000099 16.00000132 20.00000165]]


In [4]:
import numpy as np

# checking analytical and numerical gradients for difference in squared function (using two variables)
def func(x, y):
    return (x-y)**2

def analytical_gradient(func, x, y):
    return 2*(x-y)

def numerical_gradient(func, x, y, h=1e-9):
    x = np.asarray(x)
    y = np.asarray(y)
    print('here1',func(x + h, y + h))
    print('here2',func(x,y))
    return (func(x + h, y) - func(x-h,y)) / h

def numerical_gradient(func, x, y, h=1e-9):
    x = np.asarray(x)
    y = np.asarray(y)
    
    grad_x = (func(x + h, y) - func(x - h, y)) / (2 * h)
    grad_y = (func(x, y + h) - func(x, y - h)) / (2 * h)

    return np.array([grad_x]), np.array([grad_y])


x = np.array([[2,4,6,8,10]])
y = np.array([[2,4,6,7,9]])
analytical_grad = analytical_gradient(func, x, y)
numerical_grad_x, numerical_grad_y = numerical_gradient(func, x, y)

# Compare the gradients
print("Analytical Gradient:\n", analytical_grad[:5])
print("\nNumerical Gradient:\n", numerical_grad_x[:5])


Analytical Gradient:
 [[0 0 0 2 2]]

Numerical Gradient:
 [[[0.         0.         0.         2.00000017 2.00000017]]]


### MSE

In [7]:
import numpy as np

# numerical and analytical gradients for MSE
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

def analytical_gradient(y_true, y_pred):
    return - 2 * (y_true - y_pred) / len(y_true)

def numerical_gradient(func, x, y, h=1e-9):
    x = np.asarray(x)
    y = np.asarray(y)
    
    grad_x = (func(x + h, y) - func(x - h, y)) / (2 * h)
    grad_y = (func(x, y + h) - func(x, y - h)) / (2 * h)

    return np.array([grad_x]), np.array([grad_y])

x_values = np.array([[2,4,6,8,10]])
x_pred = np.array([[2,4,6,7,9]])
analytical_grad = analytical_gradient(x_values, x_pred)
numerical_grad_x, numerical_grad_y = numerical_gradient(mse, x_values, x_pred)

# Compare the gradients
print("Analytical Gradient:\n", analytical_grad[:5])
print("\nNumerical Gradient:\n", numerical_grad_y[:5])
print("\nMean of Numerical and Analytical Gradients: ",numerical_grad_y.mean(), analytical_grad.mean(),)

Analytical Gradient:
 [[ 0.  0.  0. -2. -2.]]

Numerical Gradient:
 [-0.80000007]

Mean of Numerical and Analytical Gradients:  -0.8000000661922968 -0.8


### Derivative for MSE or ERROR in our landing game are the same

In [8]:
class MSE:
    def forward(self, y_true, y_pred):
        self.loss = np.mean((y_true - y_pred)**2)
        return self.loss
    
    def backward(self, y_true, y_pred):
        self.dloss = - 2 * (y_true - y_pred) / len(y_true)
        return self.dloss
    
    
class Error:
    def forward(self, y_true, y_pred):
        self.loss = (y_true - y_pred)
        return np.mean(self.loss), -self.loss
    
    def backward(self):
        self.dloss = - self.loss
        return self.dloss
    
    
mse = MSE()
error = Error()

# for our case the gradients of mse and error are always same, because * and len(y_true) cancels each other
y_true = np.array([8.0, 13.0])
y_pred = np.array([2.2, 3.3])

mse_loss = mse.forward(y_true, y_pred)
error, grad = error.forward(y_true, y_pred)
mse_dloss = mse.backward(y_true, y_pred)

mse_dloss, grad 

(array([-5.8, -9.7]), array([-5.8, -9.7]))

### RMSE

In [9]:
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

def analytical_gradient_rmse(y_true, y_pred):
    denominator = np.sqrt(np.mean((y_true - y_pred)**2, axis=-1))
    return - (y_true - y_pred) / (len(y_true) * denominator+ np.finfo(float).eps)

def numerical_gradient_rmse(func, x, y, h=1e-9):
    x = np.asarray(x)
    y = np.asarray(y)
    
    grad_x = (func(x + h, y) - func(x - h, y)) / (2 * h)
    grad_y = (func(x, y + h) - func(x, y - h)) / (2 * h)

    return np.array([grad_x]), np.array([grad_y])

# Test
x_values = np.array([[2, 4, 6, 8, 10]])
x_pred = np.array([[-1, -2.4, -5.1, -7.6, 9.8]])

# RMSE and Analytical Gradient for RMSE
rmse_value = rmse(x_values, x_pred)
analytical_grad_rmse = analytical_gradient_rmse(x_values, x_pred)
numerical_grad_x_rmse, numerical_grad_y_rmse = numerical_gradient_rmse(rmse, x_values, x_pred)

# Compare the results
print("\nAnalytical Gradient Mean:", analytical_grad_rmse.mean())
print("Numerical Gradient Mean wrt y:", numerical_grad_y_rmse)



Analytical Gradient Mean: -0.7953854235586079
Numerical Gradient Mean wrt y: [-0.79538509]


### Proving the derivatives for Dense Layer

In [14]:
x = np.array([2.0, 4.0])
obj = DenseLayer(2,2)
obj.forward(x)
print(obj.w, obj.x) # values of x and w in forward pass

[[ 0.14940791 -0.02051583]
 [ 0.03130677 -0.08540957]] [2. 4.]


In [15]:
print(obj.backward(1), obj.dw) # values for dx and dw in backward pass

[[ 0.14940791  0.03130677]
 [-0.02051583 -0.08540957]] [2. 4.]


In [17]:
# forward and backward pass for sigmoid function
x = np.array([0])
act = Sigmoid(lmbda=1.0)
act.forward(x)

array([0.5])

In [18]:
act.backward(1)

array([0.25])